In [3]:
#!pip install web3

In [9]:
import pandas as pd
import requests
import datetime
import time
from web3 import Web3 as web3
import json

# Top token addresses
retrieve top 300 tokens addresses from CoinGekco public API per date 16. April

In [4]:
# First request to retrieve the top 100 coins by market cap
url = 'https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=1&sparkline=false'
response = requests.get(url)
data1 = response.json()

# Get the last coin in the first request
last_coin = data1[-1]

# Second request to retrieve the next 100 coins by market cap
url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=2&sparkline=false&market_cap_max={last_coin['market_cap']-1}"
response = requests.get(url)
data2 = response.json()

# Get the last coin in the second request
last_coin = data2[-1]

# Third request to retrieve the next 100 coins by market cap
url = f"https://api.coingecko.com/api/v3/coins/markets?vs_currency=usd&order=market_cap_desc&per_page=100&page=3&sparkline=false&market_cap_max={last_coin['market_cap']-1}"
response = requests.get(url)
data3 = response.json()

# Combine the three sets of data into a single list
data = data1 + data2 + data3

# Create a DataFrame from the combined data
top_300_coins = pd.DataFrame(data, columns=['name', 'symbol', 'id', 'market_cap'])

# Remove duplicate IDs, keeping the row with the highest market capitalization
top_300_coins = top_300_coins.sort_values('market_cap', ascending=False).drop_duplicates(subset=['id']).sort_index()


In [5]:
top_300_coinsd

,name,symbol,id,market_cap
0,Bitcoin,btc,bitcoin,558180095735
1,Ethereum,eth,ethereum,226900478601
2,Tether,usdt,tether,81952462366
3,BNB,bnb,binancecoin,50895777870
4,USD Coin,usdc,usd-coin,30308677379
...,...,...,...,...
295,Songbird,sgb,songbird,93819843
296,Everscale,ever,everscale,93601326
297,COTI,coti,coti,93190901
298,Steem,steem,steem,91304516


In [11]:
def get_token_addresses(tokens):
    addresses = {}
    for token in tokens:
        # set up API endpoint
        url = f'https://api.coingecko.com/api/v3/coins/{token}?localization=false'
        time.sleep(4)

        # make API call and retrieve response
        response = requests.get(url)

        # extract token address from response
        if response.status_code == 200:
            data = response.json()
            if 'ethereum' in data['platforms']:
                token_address = data['platforms']['ethereum']
                addresses[token] = token_address
                print(f"The {token} token address on Ethereum is {token_address}")
            else:
                print(f"{token} is not available on Ethereum")
        else:
            print(f"Error: {response.status_code} - {response.json()['error']}")
    
    # convert addresses to a DataFrame
    top_300_tokens_addresses = pd.DataFrame(addresses.items(), columns=['coingecko_id', 'token_address'])
    return top_300_tokens_addresses



In [9]:
token_ids = top_300_coins['id'].tolist()
top_300_tokens_addresses = get_token_addresses(token_ids)

In [12]:
top_300_tokens_addresses = get_token_addresses(token_ids)

bitcoin is not available on Ethereum
ethereum is not available on Ethereum
The tether token address on Ethereum is 0xdac17f958d2ee523a2206206994597c13d831ec7
The binancecoin token address on Ethereum is 0xb8c77482e45f1f44de1745f52c74426c631bdd52
The usd-coin token address on Ethereum is 0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48
ripple is not available on Ethereum
cardano is not available on Ethereum
The staked-ether token address on Ethereum is 0xae7ab96520de3a18e5e111b5eaab095312d7fe84
dogecoin is not available on Ethereum
The matic-network token address on Ethereum is 0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0
solana is not available on Ethereum
polkadot is not available on Ethereum
tron is not available on Ethereum
litecoin is not available on Ethereum
The binance-usd token address on Ethereum is 0x4fabb145d64652a948d72533023f6e7a623c7c53
The shiba-inu token address on Ethereum is 0x95ad61b0a150d79219dcf64e1e6cc01f0b64c4ce
avalanche-2 is not available on Ethereum
The dai token addr

KeyboardInterrupt: 

In [5]:
#adding other tokens outside of top 300 (MUTE, PERP)
token_ids_add = ['perpetual-protocol','mute']
top_300_tokens_addresses_addOns = get_token_addresses(token_ids_add)


The perpetual-protocol token address on Ethereum is 0xbc396689893d065f41bc2c6ecbee5e0085233447
The mute token address on Ethereum is 0xa49d7499271ae71cd8ab9ac515e6694c755d400c


In [19]:
top_tokens_addresses_all = top_300_tokens_addresses.append(top_300_tokens_addresses_addOns, ignore_index=True)

In [119]:
top_tokens_addresses_all = top_tokens_addresses_all.loc[top_tokens_addresses_all['token_address'].notna()]


## Retrieve price data
the tokens beginning from 2020-01-01 to 2023-04-30

In [31]:
token_addresses = token_list['token_address'].tolist()
vs_currencies = ['usd'] # Replace with the currencies you want
days_per_request = 90 # Maximum number of days per API request
start_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2023-04-30', '%Y-%m-%d')

# Format the start and end dates for the API request
start_date_str = start_date.strftime("%d-%m-%Y")
end_date_str = end_date.strftime("%d-%m-%Y")

# Make the API requests
price_data = {}
api_calls = 0
for token_address in token_addresses:
    try:
        price_data[token_address] = {}
        current_date = start_date
        while current_date < end_date:
            days = min((end_date - current_date).days, days_per_request)
            print(days)
            url = f"https://api.coingecko.com/api/v3/coins/ethereum/contract/{token_address}/market_chart/range?vs_currency={','.join(vs_currencies)}&from={current_date.strftime('%s')}&to={(current_date + datetime.timedelta(days=days)).strftime('%s')}"
            response = requests.get(url)
            response_json = response.json()
            api_calls += 1
            print(f"API call {api_calls} made")
            for price in response_json['prices']:
                timestamp = price[0]
                price_value = price[1]
                date_str = datetime.datetime.fromtimestamp(timestamp/1000).strftime("%Y-%m-%d")
                price_data[token_address][date_str] = price_value
            current_date += datetime.timedelta(days=days)
            time.sleep(8) 
    except KeyError:
        print(f"No price data found for token address {token_address}. Skipping...")
        continue


90
API call 1 made
90
API call 2 made
90
API call 3 made
90
API call 4 made
90
API call 5 made
90
API call 6 made
90
API call 7 made
90
API call 8 made
90
API call 9 made
90
API call 10 made
90
API call 11 made
90
API call 12 made
90
API call 13 made
14
API call 14 made
90
API call 15 made
90
API call 16 made
90
API call 17 made
90
API call 18 made
90
API call 19 made
90
API call 20 made
90
API call 21 made
90
API call 22 made
90
API call 23 made
90
API call 24 made
90
API call 25 made
90
API call 26 made
90
API call 27 made
14
API call 28 made
90
API call 29 made
90
API call 30 made
90
API call 31 made
90
API call 32 made
90
API call 33 made
90
API call 34 made
90
API call 35 made
90
API call 36 made
90
API call 37 made
90
API call 38 made
90
API call 39 made
90
API call 40 made
90
API call 41 made
14
API call 42 made
90
API call 43 made
90
API call 44 made
90
API call 45 made
90
API call 46 made
90
API call 47 made
90
API call 48 made
90
API call 49 made
90
API call 50 made
90
API ca

## add eth price as well


In [ ]:
# Define the currencies for which to fetch price data
vs_currencies = ['usd'] # Add any other currencies you want

# Maximum number of days per API request
days_per_request = 90 

# Define the start and end dates for the price data
start_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d') + datetime.timedelta(days=1) # Start from the day after the old end date
end_date = datetime.datetime.strptime('2023-05-01', '%Y-%m-%d') # New end date

# Format the start and end dates for the API request
start_date_str = start_date.strftime("%d-%m-%Y")
end_date_str = end_date.strftime("%d-%m-%Y")

# Make the API requests
api_calls = 0
price_data = {} # Initialize a dictionary to store the price data

# Load the existing price data from the file into a dictionary
with open('price_data_top_300.json', 'r') as f:
    price_data = json.load(f)

# Fetch the price data for Ethereum (ETH) in USD
token_address = 'ethereum'
try:
    current_date = start_date
    while current_date < end_date:
        days = min((end_date - current_date).days, days_per_request)
        print(f"price data for {token_address} from {current_date.date()} to {current_date.date() + datetime.timedelta(days=days-1)}")
        url = f"https://api.coingecko.com/api/v3/coins/ethereum/market_chart/range?vs_currency={','.join(vs_currencies)}&from={current_date.strftime('%s')}&to={(current_date + datetime.timedelta(days=days-1)).strftime('%s')}"
        response = requests.get(url)
        response_json = response.json()
        api_calls += 1
        time.sleep(4)
        for price in response_json['prices']:
            timestamp = price[0]
            price_value = price[1]
            date_str = datetime.datetime.fromtimestamp(timestamp/1000).strftime("%Y-%m-%d")
            price_data.setdefault("ether", {})[date_str] = price_value # Add data to existing dictionary
        current_date += datetime.timedelta(days=days)
  
except KeyError as e:
    print(f"No price data found for token address {token_address}. Skipping...")
    print(f"Error message: {e}")



In [9]:
price_data_top_300 = pd.DataFrame([(token_address, timestamp, price)
                   for token_address, values in price_data_top_300.items()
                   for timestamp, price in values.items()],
                  columns=['token_address', 'timestamp', 'price'])

print(price_data_top_300)

                                     token_address   timestamp        price
0       0xdac17f958d2ee523a2206206994597c13d831ec7  2020-01-01     0.999814
1       0xdac17f958d2ee523a2206206994597c13d831ec7  2020-01-02     0.999125
2       0xdac17f958d2ee523a2206206994597c13d831ec7  2020-01-03     1.001340
3       0xdac17f958d2ee523a2206206994597c13d831ec7  2020-01-04     0.998244
4       0xdac17f958d2ee523a2206206994597c13d831ec7  2020-01-05     0.999578
...                                            ...         ...          ...
160890                                       ether  2023-04-25  1869.782581
160891                                       ether  2023-04-26  1864.342719
160892                                       ether  2023-04-27  1912.383742
160893                                       ether  2023-04-28  1899.344706
160894                                       ether  2023-04-29  1905.817484

[160895 rows x 3 columns]


In [12]:
#price_data_top_300.to_csv('price_data.csv', index=False)

## add missing date values for ether price 

In [7]:
import pandas as pd
import numpy as np

# Load the DataFrame from your data source
price_data = pd.read_csv('price_data.csv')

# Filter data to only include rows where token_address = "ether"
ether_data = price_data[price_data['token_address'] == 'ether']

# Convert the timestamp column to a datetime object
ether_data['timestamp'] = pd.to_datetime(ether_data['timestamp'])

# Set the timestamp column as the index
ether_data.set_index('timestamp', inplace=True)

# Find the missing dates
missing_dates = pd.date_range(start=ether_data.index.min(), end=ether_data.index.max()).difference(ether_data.index)

# Fill in missing prices with the mean value from the day before and after the missing date
for missing_date in missing_dates:
    before = ether_data.loc[missing_date - pd.Timedelta(days=1)]
    after = ether_data.loc[missing_date + pd.Timedelta(days=1)]
    mean_price = np.mean([before['price'], after['price']])
    ether_data.loc[missing_date] = ['ether', mean_price]

# Reset the index to turn the timestamp column back into a regular column
ether_data.reset_index(inplace=True)


<ipython-input-7-7cd3f26d8408>:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  ether_data['timestamp'] = pd.to_datetime(ether_data['timestamp'])
/opt/conda/lib/python3.8/site-packages/pandas/core/indexing.py:670: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value)


In [9]:
# Merge the filled ether_data dataframe with the original price_data dataframe
filled_data = pd.concat([ether_data, price_data[price_data['token_address'] != 'ether']])

# Convert the timestamp column to a datetime object
filled_data['timestamp'] = pd.to_datetime(filled_data['timestamp'])

# Sort the data by timestamp
filled_data.sort_values('timestamp', inplace=True)

# Reset the index
filled_data.reset_index(drop=True, inplace=True)


In [14]:
#filled_data.to_csv('price_data.csv', index=False)

## add price data for MUTE and PERP
I'll include these to tokens four the analysis despite them not being in the top 300 at the time of writing this, as they constitute prominent players in the layer 2 space

In [10]:
top_tokens_addresses = pd.read_csv('top_tokens_addresses.csv')

top_tokens_addresses_AddOns = top_tokens_addresses[
    (top_tokens_addresses['token_address'] == '0xbc396689893d065f41bc2c6ecbee5e0085233447') |
    (top_tokens_addresses['token_address'] == '0xa49d7499271ae71cd8ab9ac515e6694c755d400c')
]

In [11]:
token_addresses = top_tokens_addresses_AddOns['token_address'].unique().tolist()
vs_currencies = ['usd'] # Replace with the currencies you want
days_per_request = 90 # Maximum number of days per API request
start_date = datetime.datetime.strptime('2020-01-01', '%Y-%m-%d')
end_date = datetime.datetime.strptime('2023-05-01', '%Y-%m-%d')

# Format the start and end dates for the API request
start_date_str = start_date.strftime("%d-%m-%Y")
end_date_str = end_date.strftime("%d-%m-%Y")

# Make the API requests
price_data = {}
api_calls = 0
for token_address in token_addresses:
    try:
        price_data[token_address] = {}
        current_date = start_date
        while current_date < end_date:
            days = min((end_date - current_date).days, days_per_request)
            url = f"https://api.coingecko.com/api/v3/coins/ethereum/contract/{token_address}/market_chart/range?vs_currency={','.join(vs_currencies)}&from={current_date.strftime('%s')}&to={(current_date + datetime.timedelta(days=days)).strftime('%s')}"
            response = requests.get(url)
            response_json = response.json()
            print(f"price data for {token_address} from {current_date.date()} to {current_date.date() + datetime.timedelta(days=days-1)}")
            api_calls += 1
            #print(f"API call {api_calls} made")
            for price in response_json['prices']:
                timestamp = price[0]
                price_value = price[1]
                date_str = datetime.datetime.fromtimestamp(timestamp/1000).strftime("%Y-%m-%d")
                price_data[token_address][date_str] = price_value
            current_date += datetime.timedelta(days=days)
            time.sleep(7)
    except KeyError:
        print(f"No price data found for token address {token_address} from {current_date.date()} to {current_date.date() + datetime.timedelta(days=days-1)}. Skipping...")
        continue


price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2020-01-01 to 2020-03-30
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2020-03-31 to 2020-06-28
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2020-06-29 to 2020-09-26
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2020-09-27 to 2020-12-25
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2020-12-26 to 2021-03-25
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2021-03-26 to 2021-06-23
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2021-06-24 to 2021-09-21
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2021-09-22 to 2021-12-20
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2021-12-21 to 2022-03-20
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2022-03-21 to 2022-06-18
price data for 0xbc396689893d065f41bc2c6ecbee5e0085233447 from 2022-06-19 to 2022-09-16
price data for 0xbc396689893d065

In [13]:
price_data_AddOns = pd.DataFrame([(token_address, timestamp, price)
                   for token_address, values in price_data.items()
                   for timestamp, price in values.items()],
                  columns=['token_address', 'timestamp', 'price'])

In [18]:
#price_data = pd.read_csv('price_data.csv') 
price_data = price_data.append(price_data_AddOns)
price_data.to_csv('price_data.csv', index=False)

## retreive the token decimals for the addresses

In [8]:
top_tokens_addresses = pd.read_csv('top_tokens_addresses.csv') 

In [123]:
from web3 import Web3, HTTPProvider
import pandas as pd
import time

web3 = Web3(HTTPProvider('https://mainnet.infura.io/v3/YOUR-PROJECT-ID'))

# ABI of the ERC20 token
abi = [{'constant': True, 'inputs': [], 'name': 'decimals', 'outputs': [{'name': '', 'type': 'uint8'}], 'payable': False, 'stateMutability': 'view', 'type': 'function'}]

# List of token addresses to retrieve decimals for (using lowercase addresses)
token_addresses = top_tokens_addresses_all['token_address'].tolist() 

# Create an empty DataFrame to store the results
top_tokens_decimals = pd.DataFrame(columns=['token_address', 'decimals'])

for address in token_addresses:
    # Convert the lowercase address to a checksum address
    checksum_address = Web3.to_checksum_address(address)

    # Load the ABI and contract address into the Web3 library
    contract = web3.eth.contract(address=checksum_address, abi=abi)

    # Retrieve the decimals of the token
    decimals = contract.functions.decimals().call()

    # Add the results to the DataFrame
    top_tokens_decimals = top_tokens_decimals.append({'token_address': address, 'decimals': decimals}, ignore_index=True)

    # Pause for 2 seconds to avoid hitting the API too frequently
    time.sleep(2)

print(top_tokens_decimals)


                                  token_address decimals
0    0xdac17f958d2ee523a2206206994597c13d831ec7        6
1    0xb8c77482e45f1f44de1745f52c74426c631bdd52       18
2    0xa0b86991c6218b36c1d19d4a2e9eb0ce3606eb48        6
3    0xae7ab96520de3a18e5e111b5eaab095312d7fe84       18
4    0x7d1afa7b718fb893db30a3abc0cfc608aacfebb0       18
..                                          ...      ...
171  0x3c4b6e6e1ea3d4863700d7f76b36b7f3d3f13e3d        8
172  0x198d14f2ad9ce69e76ea330b374de4957c3f850a        6
173  0xff56cc6b1e6ded347aa0b7676c85ab0b3d08b0fa       18
174  0xbc396689893d065f41bc2c6ecbee5e0085233447       18
175  0xa49d7499271ae71cd8ab9ac515e6694c755d400c       18

[176 rows x 2 columns]


In [ ]:
top_tokens_addresses= pd.merge(top_tokens_addresses, top_tokens_decimals)

In [130]:
 #top_tokens_addresses.to_csv('top_tokens_addresses.csv', index=False)